# Neural Network
***
**Name**: Nicholas Renninger
***

## Goal

The goal of this assignment is to build a one-hidden-layer back propagation network to process real data.  For this assignment you will implement the neural net (activation function and training code) yourself, not using tensorflow or other deep learning frameworks. The purpose is for you to understand the nitty gritty of what these frameworks are doing for you before we switch over to using them.

## Data 
***

In this assignment you will be using the Occupancy Detection data set (ODD). It consists of experimental data used for binary classification of room occupancy (i.e., room is occupied versus empty) based on temperature, humidity, light, and CO2 sensors. The train and test data sets are each collected over a week period.

The data set includes time stamps with date and hour/minute/second within the day. You are **not to use time stamp features** for predicting occupancy. Since this is a commercial office building, the time stamp is a strong predictor of occupancy. Rather, the goal is to determine whether occupancy - **occ** can be sensed from:
1. temperature, expressed in degrees Celsius - **temp**
2. relative humidity, expressed as a % - **hum**
3. light, in lux - **light**
4. CO2, in ppm - **cdx**
5. humidity ratio, which is derived from the temperature and the relative humidity - **hum_ratio**

In [2]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split

In [62]:
data = pd.read_csv('data/datatraining.txt')
X = data[['date', 'Humidity', 'Light', 'CO2', 'HumidityRatio', 'Temperature']].values
y = data['Occupancy'].values

X_train, X_test, y_train, y_test = train_test_split(X[:, 1:], y,
                                                    test_size=0.1)
y_train = np.expand_dims(y_train, axis=1)

### 1. Perceptron
***

Using the perceptron code you wrote for Assignment 1, train a perceptron (linear activation function with a binary threshold) using the training set. Your perceptron should have the 5 input variables described above.

**Part A**: Report the training and test set performance in terms of % examples classified correctly after 100 epochs.

**NOTE**: The Perceptron Update Rule is guaranteed to converge only if there is a setting of the weights that will classify the training set perfectly.  (The learning rule minimizes mistakes. When all examples are classified correctly, the weights stop changing.)  With a noisy data set like this one, the algorithm may not terminate.  Also remember that the perceptron algorithm is performing stochastic gradient descent. Thus, it will jitter around the ideal solution continually changing the weights from one iteration to the next. The weight changes will have a small effect on performance, so you'll see training set performance jitter a bit as well.

In [67]:
def perceptron_update(X, y, w, alpha):
    """
    One epoch of Perceptron updates (full sweep of the dataset).
    
    Parameters
    ----------
    X : NumPy array of features (size : no of examples X features)
    y : Numpy array of class labels (size : no of examples X 1)
    w : array of coefficients from the previous iteration
    alpha : Learning rate
    
    Returns
    -------
    w : Coefficients of the classifier (after updating)
    incorrect : Incorrectly classified examples
                (tuple of features, predicted label)
    accuracy: ratio of correctly classified to total examples
    """
    
    N = len(y)
    numberClassifiedRight = N
    incorrect = []
    
    for i in range(N):
        
        curr_x = X[i, :].reshape((-1, 1))
        curr_y = y[i]
        y_predicted = np.sign(w.T @ curr_x)
        
        # use perceptron update rule
        if curr_y != y_predicted:
            
            w = np.add(w, alpha * curr_y * curr_x)

            numberClassifiedRight -= 1
            incorrect.append((curr_x, y_predicted))
        
    accuracy = numberClassifiedRight / N

    return w, incorrect, accuracy

def perceptron(X, y, maxIter, alpha):
    """
    Implements the Perceptron algorithm.
    
    Parameters
    ----------
    X : NumPy array of features (size : no of examples X features)
    y : Numpy array of class labels (size : no of examples X 1)
    maxIter : The maximum number of iterations allowed 
    alpha : Learning Rate
    
    Returns
    -------
    w : Coefficients of the classifier
    incorrect : Incorrectly classified examples on termination
    accuracyPerEpoch : array-like of best accuracy each epoch
    """
    
    # as we want to learn the bias term as well, need to augment our feature
    # matrix
    X_b = getFeaturesWithBias(X)

    N = len(y)
    numFeatures = X_b.shape[1]

    # initialize gradient descent with weights all being 0
    w = np.zeros((numFeatures, 1))
    numEpochs = 0
    accuracyPerEpoch = []
    maxAccuracy = 0
    best_weights = w
    while (numEpochs < maxIter):

        w, incorrect, accuracy = perceptron_update(X_b, y, w, alpha)
        accuracyPerEpoch.append(accuracy)
        
        if accuracy > maxAccuracy:
            maxAccuracy = accuracy
            best_weights = w

        maxAccuracy = accuracy
        if accuracy == 1:
            break
        
       # slow down printing of loss
        if (numEpochs % 10) == 0:
            print('Epoch: ', numEpochs, 'accuracy: ', accuracy)
        
        numEpochs += 1
    
    print('Max Epochs: ', numEpochs, 'Max Accuracy: ', maxAccuracy)
        
    return best_weights, incorrect, accuracyPerEpoch

def getFeaturesWithBias(X):
    """
    Returns the Feature Matrix with a column of ones added for the bias term
    
    need to add a column of ones to the feature matrix to account for the
    bias term 'b' in our linear model:
            y = w_1 * x_1 + w_2 * x_2 + b
    
    Parameters
    ----------
    X : NumPy array of features (size : no of examples X features)
    
    Returns
    -------
    X_b : NumPy array of features with bias column added 
          (size : no of examples X (features + 1))
          
    Examples
    --------
    X = [[0.3 0.5]
         [0.5 0.4]
         [0.7 1.5]]
         
         
    X_b = [[0.3 0.5 1.0]
           [0.5 0.4 1.0]
           [0.7 1.5 1.0]]
    """
    
    numFeatures, _ = np.shape(X)
    biasFeatureMat = np.ones(shape=(numFeatures, 1))
    X_b = np.concatenate((X, biasFeatureMat), axis=1)
    
    return X_b

# need negative classes to be -1
y_perceptron_train = [-1 if label == 0 else 1 for label in y_train]
y_perceptron_test = [-1 if label == 0 else 1 for label in y_test]

(weights,
 incorrect,
 accuracyPerEpoch) = perceptron(X_train, y_perceptron_train, 100, alpha=0.001)

print("Perceptron weight vector:", weights)
print("Number of incorrectly classified examples:", len(incorrect))

Epoch:  0 accuracy:  0.8748635371179039
Epoch:  10 accuracy:  0.9508733624454149
Epoch:  20 accuracy:  0.9564683406113537
Epoch:  30 accuracy:  0.9632914847161572
Epoch:  40 accuracy:  0.9641102620087336
Epoch:  50 accuracy:  0.9664301310043668
Epoch:  60 accuracy:  0.9694323144104804
Epoch:  70 accuracy:  0.9683406113537117
Epoch:  80 accuracy:  0.9690229257641921
Epoch:  90 accuracy:  0.9705240174672489
Max Epochs:  100 Max Accuracy:  0.9703875545851528
Perceptron weight vector: [[-13.550079333334928]
 [3.1532666666667826]
 [1.918574999998727]
 [-0.0037039097781440206]
 [-71.90067600000029]
 [-3.013999999999779]]
Number of incorrectly classified examples: 217


The test vs. training set performances are shown below. Very good generalization :)

In [72]:
def classify(X, y, w):
    """
    Use this function to classify examples in the test set
    
    Parameters
    ----------
    X : Test set features
    y : Test set labels
    w : Perceptron coefficients
    
    Returns
    -------
    accuracy: ratio of correctly classified to total examples
    """
    
    N = len(y)
    numberClassifiedRight = N
    
    # as we want to learn the bias term as well, need to augment our feature
    # matrix
    X_b = getFeaturesWithBias(X)
    
    for i in range(N):
        
        curr_x = X_b[i, :].reshape((-1, 1))
        curr_y = y[i]

        y_predicted = np.sign(w.T @ curr_x)
        if curr_y != y_predicted:
            numberClassifiedRight -= 1

    accuracy = numberClassifiedRight / N
    
    return accuracy

# testing the perceptron
train_accuracy = classify(X_train, y_perceptron_train, weights)
test_accuracy = classify(X_test, y_perceptron_test, weights)

print('train accuracy:', train_accuracy)
print('test accuracy:', test_accuracy)

train accuracy: 0.9578329694323144
test accuracy: 0.9533742331288344


### 2. 1-layer feedforward neural net
***

In this part, you will implement a 1-layer feedforward neural network from scratch as shown in the figure below.
<img src="./res/mlp_ann.png" alt="mlp_ann" style="width:500px;"/>
Your tasks are as follows:

**PART 2.A**

1. Implement `sigmoid(x)`: In this assignment, we will be using the sigmoid activation function as the non-linearity.
2. Implement `forward(x)` which calculates the activations at layer 1, 2 and 3 (stored in variables `_a01`, `_a12`, `_a23` respectively) and returns the final layer activations.
3. Implement `backward(y, y_hat)` such that it returns layer 1 and layer 2 gradients using the mean squared loss between the predicted and groundtruth vectors.
4. Implement `update(l1_grad, l2_grad)` function which will update the weights and biases stored in `_W1, _W2 and _b1, _b2`.
5. Implement`train(X, y, epochs)`.
6. Implement `predict(X)` to return one-hot encoded representation of the `_a23` layer activation.

In [ ]:
rndg = np.random.RandomState(seed=0)

class MLP(object):
    """
        -- Multi-Layer Perceptron Neural Net.
           Network has 1 input layer followed by a hidden layer and
           an output layer. 
    """
    def __init__(self, n_vis=5, n_hid=10, n_out=1, batch_size=7, lr=0.1):
        """
            Initialize an MLP object.
            
            params
            
            n_vis (int): # of neurons in input layer.
            n_hid (int): # of neurons in hidden layer.
            n_out (int): # number of neurons in final output layer.
            batch_size (int): # number of X,y instances in a mini-batch.
            lr (double): learning rate/step size.

            class variables:
            
            _W1, _b1 (np.ndarray): 
                Weights and biases of the input layer respectively.
            
            _W2, _b2 (np.ndarray):
                Weights and biases of the hidden layer respectively.
            
            _a01, _a12, _a23 (np.ndarray or None):
                Activations from input, hidden and output layer respectively. 
                
            _l1_grad_b1, _l2_grad_b2 (np.ndarray or None):
                Gradients of the loss w.r.t. the biases. 
        """
        self._n_vis = n_vis
        self._n_hid = n_hid
        self._n_out = n_out
        
        self._W1 = rndg.normal(size=(self._n_hid, self._n_vis))
        self._b1 = np.ones((self._n_hid, 1))

        self._W2 = rndg.normal(size=(self._n_out, self._n_hid))
        self._b2 = np.ones((self._n_out, 1))

        self._a01 = None
        self._a12 = None
        self._a23 = None
        
        self._l1_grad_b1 = None
        self._l2_grad_b2 = None
        
        self.batch_size = batch_size
        self.lr = lr

    def sigmoid(self, X):
        """
            Sigmoid Logistic Function.
            
            params:
            
            X (np.ndarray)
            
            return:
            
            result (np.ndarray): result = f(X) 
                where f is the sigmoid function.
            
        """
        
        return 1 / (1 + np.exp(-X)) 
    
    def sigmoid_gradient(self, X):
        """
            Gradient of the Sigmoid Logistic Function.
            
            params:
            
            X (np.ndarray)
            
            return:
            
            result (np.ndarray): result = ∇f(X) 
                where ∇f is the gradient of the sigmoid function.
            
        """
        
        return self.sigmoid(X) @ (1.0 - self.sigmoid(X))
    
    def compute_MSE_loss(self, y, y_hat):
        """
        Computes the total MSE loss for across all y-y_hat pairs

        Parameters
        ----------
        y : Numpy array of output value 'y' (size : no of examples X 1)
        y_hat : Numpy array of estimated output value 
                (size : no of examples X 1)

        Returns
        -------
        loss : scalar MSE loss over the whole data set
        """

        N = len(y)

        # here, accounting for the 1 degree of freedom in a linear model with one
        # predictor variable for the computation of Mean-square-error (MSE))
        residuals = (y - y_hat) ** 2
        loss = 1 / (N - 2) * np.sum(residuals)

        return loss
    
    def compute_MSE_loss_gradient(self, y, y_hat):
        """
        Computes the total MSE loss gradient w.r.t. y across all y-y_hat
        pairs

        Parameters
        ----------
        y : Numpy array of output value 'y' (size : no of examples X 1)
        y_hat : Numpy array of estimated output value
                (size : no of examples X 1)

        Returns
        -------
        loss_gradient : scalar MSE loss gradient over the whole data set
        """

        N = len(y)

        # here, accounting for the 1 degree of freedom in a linear model with one
        # predictor variable for the computation of Mean-square-error (MSE))
        residuals = (y - y_hat)
        loss = 2 / (N - 2) * np.sum(residuals)

        return loss

    def forward(self, X):
        """
            Feed forward an input through each 
            of the layers of network.
            
            Store each layer activation in the class 
            variables defined in the constructor. You'll
            need them later during backpropogation.
            
            params:
            
            X (np.ndarray): batch_size x 5 dimensional array
            
            return:
            
            y_hat (np.ndarray): 1 x batch_size dimensional array
                representing final layer outputs.
            
        """
        
        self._a01 = self.sigmoid(X)
        self._a12 = self.sigmoid(self._W1 @ self._a01 + self._b1)
        self._a23 = self.sigmoid(self._W2 @ self._a12 + self._b2)
        
        return self._a23 

    def backward(self, y, y_hat):
        """
            Implement the backpropogation algorithm.
            Assume mean squared loss at the output.
            calculate and update gradients of the loss w.r.t. the biases
            
            params:
            
            y (np.ndarray): batch_size x 1 dimensional vector
                of grounf truth labels.
            y_hat (np.nd_array): 1 x 7 dimensional vector
                of predicted lables from forward().
            
            return:
            
            l1_grad (np.ndarray): n_hid x n_vis dimensional array
                representing gradients for hidden layer. 
                (gradients of the loss w.r.t. the _W1)
                
            l2_grad (np.ndarray): 1 x n_hid dimensional array
                representing gradients for input layer. 
                (gradients of the loss w.r.t. the _W2)
            
        """
        
        grad_L_output = self.sigmoid_gradient(self._a23) @ \
                        self.
        
    def update(self, l1_grad, l2_grad):
        """
            Implement the update rule for network weights and biases.
            
            params:
            
            l1_grad (np.ndarray): gradients for input layer.
            l2_grad (np.ndarray): gradients for hidden layer.
            
            return:
            
            none.
        """
        # TO-DO
        
        raise NotImplementedError()
        
    def predict(self, X, threshold = 0.5):
        """
            Returns one hot encoding of vector X using 0.5
            as the ddefault threshold.
            
            params:
            
            X (np.ndarray): predictions/activations of the output layer.
            
            return:
            
            y (np.ndarray): one hot encoding of output layer.
        """
        
        raise NotImplementedError()
    
    def train(self, X, y, epochs):
        """
            Implement the train loop for n epochs.
            In each epoch do the following:
            1. Shuffle the dataset.
            2. create self.batch_size sized-mini-batches of the dataset.
            3. get network predictions using forward().
            4. calculate the gradients using backward().
            5. update the network weights using update().
            6. Repeat 1-5 until convergence.
            
            params:
            
            X (np.ndarray): N x 5 dimensional ndarray of inputs.
            y (np.ndarray): N x 1 dimensional array of true labels.
            epochs (int): # of epochs to train the network for. 
            
            return:
            
            none.
        """
        # TO-DO
        
        raise NotImplementedError()
            
    def test(self, X_test, y_test):
        """
            Implement the test function which reports accuracy of 
            the model on the test set X_test against the true labels
            y_test.
            
            You may re-use the predict and accuracy functions defined above.
            
            params:
            
            X_test (np.ndarray): N x M dimensional array where M is the 
                number of attributes considered for training the model.
            y_test (np.ndarray): N x 1 dimensional array.
            
            return:
            
            accuracy (double): accuracy of the predicted labels against the 
                groundtruth labels.
        """
        
        raise NotImplementedError()
    
    @classmethod
    def accuracy(self, y, y_hat):
        len_ = y.flatten().shape[0]
        return np.sum(y.flatten() == y_hat.flatten())/len_

    

In [ ]:
import unittest
from sklearn.datasets import make_classification

def load_params(path, model):
    
    params = pickle.load(open(path, 'rb'))
    for key in params:
        model.__dict__[key] = params[key]

class TestMLP(unittest.TestCase):
    def setUp(self):
        
        self.net = MLP()
        self.pre_trained_model = load_params("./res/mlp.pkl", self.net)
        self.X = np.load("./res/x.npy")
        self.y = np.load("./res/y.npy")
        self.p = np.load("./res/p.npy")
        self.sigX = np.asarray([-1, -2, -3, 0, 1, 2, 3])
    
    def test_forward(self):
        self.assertEqual(np.sum(self.net.forward(self.X[901:])),
                         np.sum(self.p))
    
    def test_train(self):
        x, y = make_classification(
            n_samples=1000,
            n_features=5,
            n_redundant=0,
            random_state=rndg
        )
        net = MLP(5, 5, 1, 7, 0.01)
        net.train(self.X[:900], self.y[:900], 100)
        self.assertTrue(net.accuracy(net.predict(self.X[901:]),
                                     self.y[901:]) > 0.88)
    
    def test_sigmoid(self):
        self.assertEqual(
            np.sum(self.net.sigmoid(self.sigX)), 3.5
        )
        
suite = unittest.TestLoader().loadTestsFromTestCase(TestMLP)
unittest.TextTestRunner().run(suite)

**PART 2.B**

Report the baseline performance of the untrained MLP that you built in **PART 2.A** using the selected values. To calculate the baseline score, feed-forward entire test set through the test function. The test function will return a set of predictions. Calculate the accuracy of these predictions against ground truth training lables.  
Report:
1. accuracy

In [6]:
### Part 2.B

**PART 2.C**

Select a learning rate, batch size and train the network for 100 epochs. Plot a graph of network's accuracy on the test data as a function of epoch. On the same graph, plot a constant horizontal line of the baseline score that you reported in **PART 2.B**.  

Report:
1. learning rate
2. batch size

In [7]:
### Part 2.C

**PART 2.D**

Now train nets with varying size of the hidden layer $H={1, 2, 5, 10..}$ for max epochs = 100. Make a plot of the nets' accuracy on test set as a function of $H$.

In [8]:
### Part 2.D

### 3. Extra Credit

See how much adding information about time of day helps the network. Add a new set of inputs that represent the time of day. (Don't add information about day of week or absolute date.)  


**PART 3.A**  

Determine an appropriate representation for the time of day. Describe the representation you used. For example, you might add one unit with a value ranging from 0 to 1 for times ranging from 00:00 to 23:59. Report the representation you selected.


In [9]:
### Part 3.A

**PART 3.B**  

Train your net with $H=5$ hidden and compare training and test set performance to the net you built in (2c)

In [10]:
### Part 3.B